In [1]:
%config Completer.use_jedi = False
%load_ext autoreload

# Imports 

In [2]:
from crawler.src import recommender

In [45]:
import pandas as pd
from scipy.sparse import coo_matrix, csr_matrix
import numpy as np
from sklearn.preprocessing import LabelEncoder
from implicit.als import AlternatingLeastSquares
from scipy import sparse

In [4]:
pd.options.display.max_columns=200
pd.options.display.max_rows=200

In [5]:
df_rank = pd.read_csv("../../data/crawler/boardgame_ranks_20250526.csv")
df_rank.head()

,id,name,yearpublished,rank,bayesaverage,average,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank,queried_at_utc
0,224517,Brass: Birmingham,2018,1,8.40146,8.57715,52086,0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2025-05-26T10:36:34
1,161936,Pandemic Legacy: Season 1,2015,2,8.36386,8.51619,55588,0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,2025-05-26T10:36:34
2,342942,Ark Nova,2021,3,8.34620,8.53657,52459,0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,2025-05-26T10:36:34
3,174430,Gloomhaven,2017,4,8.32586,8.56341,64874,0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,2025-05-26T10:36:34
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.22914,8.57682,26087,0,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,2025-05-26T10:36:34


In [45]:
df_rank.shape

(165498, 17)

In [46]:
df_rank.loc[df_rank["is_expansion"]==0].shape

(129576, 17)

# Load parquet file

In [43]:
df_bg_data_raw = pd.read_parquet("../../data/boardgame_data/boardgame_data_raw_1748311285.parquet")
print(df_bg_data_raw.shape)
df_bg_data_raw.head()

(42486, 36)


,game_id,thumbnail,image,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,boardgamecategory,boardgamemechanic,boardgamefamily,boardgameexpansion,boardgameartist,boardgamecompilation,boardgameimplementation,boardgamedesigner,boardgamepublisher,boardgameintegration,stddev,median,averageweight,owned,trading,wanting,wishing,numcomments,numweights,numratings,suggested_playerage,suggested_playerage_quartiles,language_dependence,language_dependence_quartiles,player_count_recs,suggested_numplayers,versions
0,13,https://cf.geekdo-images.com/PyUol9QxBnZQCJqZI...,https://cf.geekdo-images.com/PyUol9QxBnZQCJqZI...,3,4,120,60,120,10,"[Economic, Negotiation]","[Chaining, Dice Rolling, Hexagon Grid, Income,...","[Animals: Sheep, Components: Hexagonal Tiles, ...","[20 Jahre Darmstadt Spielt, Brettspiel Advents...","[Volkan Baga, Tanja Donner, Pete Fenlon, Jason...","[CATAN 3D Collector's Edition, Catan: 25 Jahre...","[Baden-Württemberg Catan, Catan Geographies: G...",[Klaus Teuber],"[KOSMOS, 64 Ounce Games, 999 Games, Albi, Asmo...",[],1.50101,0.0,2.2871,223336,2306,519,7567,22919,8360,135593,"{'total_votes': 577, '2': 1, '3': 0, '4': 2, '...","{'50 percent': '8', '25 percent': '8', '75 per...","{'total_votes': 346, '1': 21, '2': 294, '3': 3...",{'75 percent': 'Some necessary text - easily m...,"{'Not Recommended': ['1', '2', '4+'], 'Recomme...","{'total_votes': 2071, '1': {'Best': 1, 'Recomm...","[{'version_id': 467634, 'version_nickname': 'A..."
1,822,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...,2,5,45,30,45,7,"[Medieval, Territory Building]","[Area Majority / Influence, Enclosure, End Gam...","[Category: Dized Tutorial, Cities: Carcassonne...","[20 Jahre Darmstadt Spielt, Antwerpen (fan exp...","[Marcel Gröber, Doris Matthäus, Anne Pätzke, C...","[Carcassonne Big Box, Carcassonne Big Box 2, C...","[The Ark of the Covenant, Carcassonne Demo, Ca...",[Klaus-Jürgen Wrede],"[Hans im Glück, 64 Ounce Games, 999 Games, Alb...","[Carcassonne: Wheel of Fortune, Mists over Car...",1.31104,0.0,1.8880,209535,2034,689,10188,22592,8466,134541,"{'total_votes': 766, '2': 2, '3': 1, '4': 12, ...","{'25 percent': '6', '75 percent': '8', '50 per...","{'total_votes': 490, '6': 481, '7': 7, '8': 1,...","{'75 percent': 'No necessary in-game text', '5...","{'Not Recommended': ['1', '5+'], 'Best': ['2',...","{'total_votes': 2659, '1': {'Best': 7, 'Recomm...","[{'version_id': 31146, 'version_nickname': 'Al..."
2,30549,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...,2,4,45,45,45,8,"[Medical, Travel]","[Action Points, Chaining, Contracts, Cooperati...","[Components: Map (Global Scale), Components: M...",[Pandemic: Gen Con 2016 Promos – Z-Force Team ...,"[Josh Cappel, Christian Hanisch, Régis Moulun,...",[],"[Fall of Rome, Iberia, Pandemic Legacy: Season...",[Matt Leacock],"[Z-Man Games, (Unknown), ADC Blackfire Enterta...",[],1.33738,0.0,2.3968,214983,3251,619,11200,20067,6189,131235,"{'total_votes': 608, '2': 0, '3': 0, '4': 1, '...","{'25 percent': '8', '75 percent': '10', '50 pe...","{'total_votes': 341, '11': 10, '12': 279, '13'...",{'75 percent': 'Some necessary text - easily m...,"{'Recommended': ['1', '2', '3'], 'Best': ['4']...","{'total_votes': 2123, '1': {'Best': 170, 'Reco...","[{'version_id': 30851, 'version_nickname': 'Al..."
3,68448,https://cf.geekdo-images.com/35h9Za_JvMMMtx_92...,https://cf.geekdo-images.com/35h9Za_JvMMMtx_92...,2,7,30,30,30,10,"[Ancient, Card Game, City Building, Civilizati...","[Closed Drafting, End Game Bonuses, Hand Manag...","[Ancient: Babylon, Ancient: Egypt, Ancient: Gr...","[7 Wonders: Armada, 7 Wonders: Armada – Siracu...","[Dimitri Chappuis, Miguel Coimbra, Etienne Heb...",[],"[7 Wonders (Second Edition), 7 Wonders Duel, 7...",[Antoine Bauza],"[Repos Production, ADC Blackfire Entertainment...",[],1.27709,0.0,2.3158,149251,1934,985,14419,16874,5414,109297,"{'total_votes': 561, '2': 0, '3': 0, '4': 1, '..."

In [44]:
df_bg_data_raw.tail()

,game_id,thumbnail,image,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,boardgamecategory,boardgamemechanic,boardgamefamily,boardgameexpansion,boardgameartist,boardgamecompilation,boardgameimplementation,boardgamedesigner,boardgamepublisher,boardgameintegration,stddev,median,averageweight,owned,trading,wanting,wishing,numcomments,numweights,numratings,suggested_playerage,suggested_playerage_quartiles,language_dependence,language_dependence_quartiles,player_count_recs,suggested_numplayers,versions
42481,39412,https://cf.geekdo-images.com/ClGWqwZDx5SJgOjh2...,https://cf.geekdo-images.com/ClGWqwZDx5SJgOjh2...,2,10,0,0,0,0,"[Expansion for Base-game, Fighting, Humor, Mat...","[Critical Hits and Failures, Dice Rolling, Gri...","[Creatures: Aliens / Extraterrestrials, Theme:...",[Macho Women with Guns: The Final Chapter (Par...,[Richard Menustik],[],[],[Greg Porter],[Blacksburg Tactical Research Center],"[Macho Women with Guns, Renegade Nuns on Wheels]",0.00000,0.0,0.0000,27,2,0,4,3,0,1,{},{},{},{},"{'Best': ['1', '2', '3', '4', '5', '6', '7', '...","{'total_votes': 0, '1': {'Best': 0, 'Recommend...",None
42482,39415,https://cf.geekdo-images.com/aXSB_LM1jYoA3a0l4...,https://cf.geekdo-images.com/aXSB_LM1jYoA3a0l4...,2,6,60,60,60,12,"[Electronic, Party Game, Trivia]",[Roll / Spin and Move],[Game: Trivial Pursuit],[],[],[],[],[(Uncredited)],"[Parker Brothers, Electronic Arts Inc. (EA), H...",[],1.84127,0.0,1.3333,488,25,1,6,46,3,142,{},{},"{'total_votes': 1, '41': 0, '42': 0, '43': 0, ...",{'75 percent': 'Unplayable in another language...,"{'Not Recommended': ['1'], 'Recommended': ['2'...","{'total_votes': 1, '1': {'Best': 0, 'Recommend...","[{'version_id': 32942, 'width': 11, 'length': ..."
42483,39417,https://cf.geekdo-images.com/k9jxqwpTuW_zTgeVl...,https://cf.geekdo-images.com/k9jxqwpTuW_zTgeVl...,2,4,0,0,0,3,"[Children's Game, Fantasy]",[Roll / Spin and Move],"[Authors: Charles Perrault, Authors: The Broth...",[],[],[],[],[(Uncredited)],"[University Games, University Games]",[],0.00000,0.0,0.0000,0,0,0,0,0,0,0,{},{},{},{},"{'Best': ['1', '2', '3', '4', '4+']}","{'total_votes': 0, '1': {'Best': 0, 'Recommend...",None
42484,39423,https://cf.geekdo-images.com/IIyLKkMcl0Vw0z0C4...,https://cf.geekdo-images.com/IIyLKkMcl0Vw0z0C4...,1,3,5,5,5,0,[Children's Game],[],[],[],[],[],[],[(Uncredited)],[Användbart Litet Företag],[],1.69967,0.0,1.0000,6,0,0,0,1,1,3,"{'total_votes': 1, '2': 1, '3': 0, '4': 0, '5'...","{'75 percent': '2', '50 percent': '2', '25 per...",{},{},"{'Best': ['1', '2', '3', '3+']}","{'total_votes': 0, '1': {'Best': 0, 'Recommend...",None
42485,39424,https://cf.geekdo-images.com/PTh678EsZZ7Gofl98...,https://cf.geekdo-images.com/PTh678EsZZ7Gofl98...,2,8,0,0,0,0,"[Action / Dexterity, Sports]",[],[Sports: Golf],[],[],[],[],[(Uncredited)],"[Parker Brothers, Parker Brothers]",[],0.00000,0.0,0.0000,1,0,0,0,0,0,0,{},{},{},{},"{'Best': ['1', '2', '3', '4', '5', '6', '7', '...","{'total_votes': 0, '1': {'Best': 0, 'Recommend...",None


In [ ]:
df_bg_data_raw

In [28]:
df_bg_data_raw.loc[df_bg_data_raw["game_id"]==2627]

,game_id,thumbnail,image,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,boardgamecategory,boardgamemechanic,boardgamefamily,boardgameexpansion,boardgameartist,boardgamecompilation,boardgameimplementation,boardgamedesigner,boardgamepublisher,boardgameintegration,stddev,median,averageweight,owned,trading,wanting,wishing,numcomments,numweights,numratings,suggested_playerage,suggested_playerage_quartiles,language_dependence,language_dependence_quartiles,player_count_recs,suggested_numplayers,versions


In [52]:
df_ratings = pd.read_parquet("../../data/boardgame_data/boardgame_rating_data_1748319041.parquet")
print(df_ratings.shape)
df_ratings.head()

(1420, 20)


,game_id,2.0,3.0,4.0,5.0,5.5,6.0,6.5,7.0,7.5,8.0,8.5,9.0,9.5,10.0,1.0,2.5,4.5,3.5,1.5
0,179350,"[obifrankenobi, cfarrell]","[Carlos Palomino, Beorne, asiok, Jetseb, Janon...","[Grildensnork, Murdockus, vetinari7878, Rabbit...","[Nico D, woj_settlers, halbert82, Chuckila, di...","[Bryo4, carontissimo, LookAtTheBacon, sillue, ...","[nunovix, Wendigohexe, zyxyv, hedisch, tgbruis...","[Orphebus, Kuula, littlebanshee, monkeykenobi,...","[MIKI1980, dbarbaglia, teamat, PlayThisToday, ...","[s-man, Mottchen, dpkmc, Surya, Pum_, LucaLeon...","[stoneghe, KABIA66, skipper2206, Turbo2000, ka...","[nunoR, liga, Quintious, manucube, avanspronse...","[Faustoxx, the_spy, Olaf78, Racing cards, dtiv...","[nintakin, nikalibi]","[Gio8, BhART, franck1912, jrubin1, jerome53240...",None,None,None,None,None
1,99808,"[gimmster, saxophonechapa, eduart, DavidMcGreg...","[Randy Cox, hkhutchi, Gyges, Kaelistus, garg, ...","[Jbrady, mmtmr, sfox, Cerebremancer, cdefrisco...","[dhailey, Grildensnork, Justinschaffer, jgoyes...","[Lothartvni, BC Indy, nittany0508, fehrmeister...","[HooeyPooey, kiddynamite, zefquaavius, Bobby4t...","[TundraThunder, JensFL, Menma, Guiannos, Mibao...","[Errante, maydaygames, kingofthegrill, mechani...","[d_hoopster, takras, fringer, jeremycoe, Conne...","[DAZHEL, MikoML, Boots01, Braidrock, nymartin2...","[mtn55, soulyogurt, eiopqrtuwy, theapostlegree...","[kmbx13, b4kabob, prosfilaes, NerdDad, kommand...",None,"[Flambers, CRRXVI, paintime, TheBardGamer]","[paulo, McCrank, Kenig]",[kcayoung],"[Spyzou, Mario Pawlowski, Zepheus, Janon, smil...",None,None
2,286667,[idontplayco_ops],"[dream spawn, aranvulnep, Janasik, realwizard,...","[raile, jonpurkis, AmandaDesignsGames, StellaH...","[povertidonki, Invicta Bricks, ryansmum2008, s...","[stroutqb22, Elpete, Duke_Fleed, rsj77, in8nir...","[feldfan2014, cabbagekingaf, Innsmouth_Look, j...","[boardingschoolbrandi, acecondor, drulive99, h...","[QuixoticQuixotic, fencedingates, AerynSpaceFa...","[TaijiJohn, Samutom84, ctrosejr, Jim1701, iuga...","[AHforever, ku8843, Wolverutto, warta, JimmyEW...","[Grenade, FrostyMeeple, sorcretic, Renno]","[RichardIngram, cardshark28800, jcadillac, cft...",None,"[matty0226, Little_Rocket_Games, alessandra198...","[ultimoratio, hazered, dakotamassie]",None,[radiofyr309],None,None
3,170041,"[scps14, Simcsa, danchi, cadhlan]","[imoo, missiloon, Sweetdogpaw, djhill, JohansG...","[StefanvM1, Livefree, leffe dubbel, Drval, woj...","[Acido, crazyclark, takokawa, river911, costel...","[Menma, Femtitala, linkme, pethulhu, Dalkuldar...","[morishita, Hanno, SplitCreek, Molotail, Great...","[rjgomes, MattCNixon, cogo71, Izobretenik, pec...","[pecotti, DeDaan, nivatra, Frontliner, Els76, ...","[tosh_loco, strubs42, gruntsudens, Bresto, epi...","[Gorkx77, deezynah, Wickedheart, s0915579408, ...","[Poeppelhoppers, vikachoo, MariBor]","[NatkaGra, pilgrim152, Tolkana, castor1966, Su...","[Rosiria, Marysinka]","[Noni1702, Racing cards, silenthoch, ThunderWo...",None,None,[Shamragnar],None,None
4,300993,[MaxRokotansky],"[MrIkerito, Geeken, Giulii, chumy, elchabonque...","[Alesbolmol, RafaGR, ckomiya, josecarloscom, a...","[Erreka, RubenMS, Santycapi, Yorkaeff, maxiscp...","[nick_arg, MissRiuku, fustero, Norg222, Hoja]","[Juan_Aguilera, thisisrhun, Braculaventura, Ev...","[AngPredator, Ninorro, darthway, Jesseaglerock...","[HANJEL, MrCaronte77, pablo24091980, sandraast...","[Misiga, facalrivas, Guanarteme80, Parsifal1, ...","[peterpax, Jowyjokes, Alex_Luka23, Charly81, E...","[bichejote, Acoidan85, herbius, Whirlpool, mru...","[jdhorux, xelerad, PabloJimenezRoman, Zelgadis...","[iwancho, robinson_2017]","[NinyaSiniestro, Cascabel93, David S Serrano, ...",None,None,"[moncsal213, RedFate]",None,None


In [53]:
df_ratings.tail()

,game_id,2.0,3.0,4.0,5.0,5.5,6.0,6.5,7.0,7.5,8.0,8.5,9.0,9.5,10.0,1.0,2.5,4.5,3.5,1.5
1415,197439,None,None,"[apocatastasi, ratbulogg, hcbftw, Bangla, whit...","[King Gary, Tikigod, KRWoods1, Peepser, Preach...","[Ephastion, StephenV, REGGY, djboss, Leo-T]","[prd1982, cfarrell, spadovsky, Michael Ward, t...","[Murdockus, Svarthand, marge2, ryanabrams, dki...","[jamomoto, frankstarro, stoneghe, e.c.r., Benj...","[slashing, LucaLeoncavallo, chg21012, BerenBar...","[Geeken, roro77, bluekingzog, hockeyking9, jan...","[jahco1980, Tuttle757, devaed, Friedrich, itha...","[PierreP, echallstrom, Snoochies, Fogtmann, sa...",[aslskfan],"[drillSGT, wtdickens, Blake the Flake, idjohns...",[Zete],None,None,None,None
1416,424981,None,None,"[himokl, gepesso, pvtparts, azeem]","[arron77, slowPigeon, Bigshowfan, Oulibanoum]",None,"[fourquartets, evil_lemons, timtims1, jmdsplot...","[Victorshade, Earl of Dukes, dragos_br]","[Neale2006, Primertt, larryjrice, truthwritten...","[Escapist_Artist, Der Gwiedo, johnnysavs, g6gr...","[MaitreYo06, Moe33, feldfan2014, BobShick, Clo...","[bionicle4365, Darla, C_Cheung, joshbalvin, sp...","[trimall, billybob0101011, oatmeal1201, marsti...","[LordNoumenon, survivorfan, Japara, zenxacred,...","[Sketcherrrr, logopolys, MelbaMo, nfirehazzard...",None,None,None,None,None
1417,352591,None,None,"[lks524, Zylofan, bubblepipemedia, CMagex, him...","[kyk10pes, mdiep, docpooh18, Catyrpelius, self...","[webstergroup, minakobrown, JoshDip]","[GizmoBoardGames, Kayma, Primori, Darrius Tuck...","[crocko33, Mkyrmse, Cuelannoo, AegisToast, Aeg...","[boardgameclinic, Rafiki_Robert, xerossilence,...","[Andorra9200, Squalidsafe, Sir Melancholie, ta...","[challahback252, FreshBao, cardboardeaux, Jess...","[wobbly_chairs, PeteWiFi]","[steph_noack, Master Thomas, ACGalaga, Desertm...","[jeremiahbrammer, Kenra]","[jamboldr, Grey_Pilgrim, seaskytowers, Scott99...",None,None,[vyvyannexp],None,None
1418,181289,None,None,None,"[Tudada, xxlmc, FrAnkX0r88, BlueLycan, ChunmaV...","[Panta_rhei, chama_81]","[failtech, Joe2016, sykano, iquito, simon055, ...",[waitmind],"[boardmathias, jgbaxter, kaddimaus, zeronemo, ...","[knotsao, Mr. Panci, High Elf Andrew, gthanato...","[damboshi, zdsarq32, netbus22, virginis, bluem...","[Spielkind78, hodoliiiii, Markus Shepherd, Hal...","[Mister_Peon, just_lis, nyravv, nimiralism, xp...","[Eurodancer, PMatheus, Ollip, fmmhide, Vermout...","[bignike_ch, jinlv81, Michiel, MarkuZa, irene0...","[justletmedownload22, Jimmy_Dean]",None,None,None,None
1419,351735,None,None,None,"[Spock71, apuity, barba, funktor, ryansdavis]",None,"[Shinn_S, wilsonza, Labbe, B2TB, Linkovich, hh...","[NICONIGLIO, lumiotso, Teurlinx, Htivante23, i...","[tribefan07, CassyCZ, fanta1995, firyfly, Coli...","[cinocinosnellman, 2goofy, Fumier, Imrhalis, P...","[danielklinge, HariSeldon86, vlcaczech, eligam...","[norberts, xerxes2k, Suspectus, Parkettwischer...","[Swoosl, Buddha007, KatsuIas, Big Fat Penguin,...","[Ema_C, loopszz]","[misskitten, heliossone, NemesisWhip, LotusTha...",[heinhunstig],None,[k4k7uz],None,None


In [19]:
df_bg_data_raw["game_id"].nunique()

11768

# Recommender testing

In [50]:
df_rank.loc[df_rank["id"].isin(df_ratings["game_id"].astype(int).tolist())].sort_index().head(50)

,id,name,yearpublished,rank,bayesaverage,average,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank,queried_at_utc
1,161936,Pandemic Legacy: Season 1,2015,2,8.36386,8.51619,55588,0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,2025-05-26T10:36:34
2,342942,Ark Nova,2021,3,8.34620,8.53657,52459,0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,2025-05-26T10:36:34
3,174430,Gloomhaven,2017,4,8.32586,8.56341,64874,0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,2025-05-26T10:36:34
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.22914,8.57682,26087,0,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,2025-05-26T10:36:34
6,167791,Terraforming Mars,2016,7,8.20027,8.34848,106321,0,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,2025-05-26T10:36:34
9,162886,Spirit Island,2017,10,8.14215,8.34133,57486,0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,2025-05-26T10:36:34
10,291457,Gloomhaven: Jaws of the Lion,2020,11,8.12757,8.39387,37838,0,NaN,NaN,NaN,NaN,NaN,11.0,7.0,NaN,2025-05-26T10:36:34
13,12333,Twilight Struggle,2005,14,8.05311,8.23676,50487,0,NaN,NaN,NaN,NaN,NaN,14.0,NaN,2.0,2025-05-26T10:36:34
14,182028,Through the Ages: A New Story of Civilization,2015,15,8.03713,8.26768,33348,0,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,2025-05-26T10:36:34
15,84876,The Castles of Burgundy,2011,16,8.02123,8.14946,65940,0,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN,2025-05-26T10:36:34


In [30]:
df_ratings = pd.read_parquet("../../data/crawler/boardgame_ratings_1748670263.parquet")
print(df_ratings.shape)

(7459, 22)


In [55]:
import importlib
importlib.reload(recommender)

<module 'crawler.src.recommender' from '/home/ubuntu/git/pax_tt_recommender/crawler/src/recommender.py'>

In [56]:
recs = recommender.GameRecommender(min_ratings_per_user=5)
recs.fit(df_ratings.set_index("game_id"))

In [76]:
game_recs = recs.recommend_similar_games(
    game_ids=[
        "337627",
        "342942",
        "84876",
        "167791",
        "358661",
        "325494",
        "356754",
        "367966",
        "363622",
        "391163",
        "416851",
        "379629",
        "357563",
        "414317",
        "364073",
        "367220", 
        "393325"
    ],
    disliked_games=["124361", "360692", "162886", "242705", "387378", "346703"],
    n_recommendations=20,
)
df_rank.loc[df_rank["id"].isin([int(x[0]) for x in game_recs]), ["name", "id"]]

,name,id
52,Cascadia,295947
57,SETI: Search for Extraterrestrial Intelligence,418059
143,Wyrmspan,410201
200,Welcome to the Moon,339789
376,The Guild of Merchant Explorers,350933
413,The Vale of Eternity,385529
491,Let's Go! To Japan,368173
520,World Wonders,365258
555,Bomb Busters,413246
587,Trio,352515


In [77]:
game_recs

[('385529', np.float64(0.06785633913924893)),
 ('411875', np.float64(0.06053934934987508)),
 ('418059', np.float64(0.05840121686665953)),
 ('420087', np.float64(0.05533289248011547)),
 ('352515', np.float64(0.052347185592146184)),
 ('368173', np.float64(0.05031468468552838)),
 ('350933', np.float64(0.050026411196026854)),
 ('365258', np.float64(0.04820333995296913)),
 ('353545', np.float64(0.047814332285280786)),
 ('413246', np.float64(0.047633915639819085)),
 ('338093', np.float64(0.04447478084648265)),
 ('339789', np.float64(0.04306313781506729)),
 ('397385', np.float64(0.042361647799737945)),
 ('406854', np.float64(0.04184043808879839)),
 ('402676', np.float64(0.041562102540840584)),
 ('403754', np.float64(0.04133019349959283)),
 ('399941', np.float64(0.04111172453914003)),
 ('410201', np.float64(0.04101625461555941)),
 ('295947', np.float64(0.04097151138347144)),
 ('384213', np.float64(0.04074862547557892))]

6      Terraforming Mars
9          Spirit Island
32              Wingspan
36               Barrage
52              Cascadia
117             Carnegie
Name: name, dtype: object

In [ ]:
# Initialize and fit the recommender
recs = GameRecommender(min_ratings_per_user=3)
recs.fit(your_dataframe)

# Get recommendations for a user
user_recommendations = recs.recommend(user_id="user123", n_recommendations=5)

# OR get similar games based on a list of games
game_recommendations = recs.recommend(game_ids=["game1", "game2"], n_recommendations=5)

In [ ]:
GameRecommender

In [5]:
df_ratings.head()

,game_id,2.0,3.0,4.0,5.0,5.5,6.0,6.5,7.0,7.5,8.0,8.5,9.0,9.5,10.0,1.0,2.5,4.5,3.5,1.5,0.0,0.5
0,179350,"[obifrankenobi, cfarrell]","[Carlos Palomino, Beorne, asiok, Jetseb, Janon...","[Grildensnork, Murdockus, vetinari7878, Rabbit...","[Nico D, woj_settlers, halbert82, Chuckila, di...","[Bryo4, carontissimo, LookAtTheBacon, sillue, ...","[nunovix, Wendigohexe, zyxyv, hedisch, tgbruis...","[Orphebus, Kuula, littlebanshee, monkeykenobi,...","[MIKI1980, dbarbaglia, teamat, PlayThisToday, ...","[s-man, Mottchen, dpkmc, Surya, Pum_, LucaLeon...","[stoneghe, KABIA66, skipper2206, Turbo2000, ka...","[nunoR, liga, Quintious, manucube, avanspronse...","[Faustoxx, the_spy, Olaf78, Racing cards, dtiv...","[nintakin, nikalibi]","[Gio8, BhART, franck1912, jrubin1, jerome53240...",None,None,None,None,None,None,None
1,99808,"[gimmster, saxophonechapa, eduart, DavidMcGreg...","[Randy Cox, hkhutchi, Gyges, Kaelistus, garg, ...","[Jbrady, mmtmr, sfox, Cerebremancer, cdefrisco...","[dhailey, Grildensnork, Justinschaffer, jgoyes...","[Lothartvni, BC Indy, nittany0508, fehrmeister...","[HooeyPooey, kiddynamite, zefquaavius, Bobby4t...","[TundraThunder, JensFL, Menma, Guiannos, Mibao...","[Errante, maydaygames, kingofthegrill, mechani...","[d_hoopster, takras, fringer, jeremycoe, Conne...","[DAZHEL, MikoML, Boots01, Braidrock, nymartin2...","[mtn55, soulyogurt, eiopqrtuwy, theapostlegree...","[kmbx13, b4kabob, prosfilaes, NerdDad, kommand...",None,"[Flambers, CRRXVI, paintime, TheBardGamer]","[paulo, McCrank, Kenig]",[kcayoung],"[Spyzou, Mario Pawlowski, Zepheus, Janon, smil...",None,None,None,None
2,286667,[idontplayco_ops],"[dream spawn, aranvulnep, Janasik, realwizard,...","[raile, jonpurkis, AmandaDesignsGames, StellaH...","[povertidonki, Invicta Bricks, ryansmum2008, s...","[stroutqb22, Elpete, Duke_Fleed, rsj77, in8nir...","[feldfan2014, cabbagekingaf, Innsmouth_Look, j...","[boardingschoolbrandi, acecondor, drulive99, h...","[QuixoticQuixotic, fencedingates, AerynSpaceFa...","[TaijiJohn, Samutom84, ctrosejr, Jim1701, iuga...","[AHforever, ku8843, Wolverutto, warta, JimmyEW...","[Grenade, FrostyMeeple, sorcretic, Renno]","[RichardIngram, cardshark28800, jcadillac, cft...",None,"[matty0226, Little_Rocket_Games, alessandra198...","[ultimoratio, hazered, dakotamassie]",None,[radiofyr309],None,None,None,None
3,170041,"[scps14, Simcsa, danchi, cadhlan]","[imoo, missiloon, Sweetdogpaw, djhill, JohansG...","[StefanvM1, Livefree, leffe dubbel, Drval, woj...","[Acido, crazyclark, takokawa, river911, costel...","[Menma, Femtitala, linkme, pethulhu, Dalkuldar...","[morishita, Hanno, SplitCreek, Molotail, Great...","[rjgomes, MattCNixon, cogo71, Izobretenik, pec...","[pecotti, DeDaan, nivatra, Frontliner, Els76, ...","[tosh_loco, strubs42, gruntsudens, Bresto, epi...","[Gorkx77, deezynah, Wickedheart, s0915579408, ...","[Poeppelhoppers, vikachoo, MariBor]","[NatkaGra, pilgrim152, Tolkana, castor1966, Su...","[Rosiria, Marysinka]","[Noni1702, Racing cards, silenthoch, ThunderWo...",None,None,[Shamragnar],None,None,None,None
4,300993,[MaxRokotansky],"[MrIkerito, Geeken, Giulii, chumy, elchabonque...","[Alesbolmol, RafaGR, ckomiya, josecarloscom, a...","[Erreka, RubenMS, Santycapi, Yorkaeff, maxiscp...","[nick_arg, MissRiuku, fustero, Norg222, Hoja]","[Juan_Aguilera, thisisrhun, Braculaventura, Ev...","[AngPredator, Ninorro, darthway, Jesseaglerock...","[HANJEL, MrCaronte77, pablo24091980, sandraast...","[Misiga, facalrivas, Guanarteme80, Parsifal1, ...","[peterpax, Jowyjokes, Alex_Luka23, Charly81, E...","[bichejote, Acoidan85, herbius, Whirlpool, mru...","[jdhorux, xelerad, PabloJimenezRoman, Zelgadis...","[iwancho, robinson_2017]","[NinyaSiniestro, Cascabel93, David S Serrano, ...",None,None,"[moncsal213, RedFate]",None,None,None,None


In [ ]:
# Step 1: Flatten and explode data
rows = []
for _, row in df_ratings.iterrows():
    for rating in df_ratings.columns:
        users = row[rating]
        game_id = row["game_id"]
        if isinstance(users, list):
            for user in users:
                rows.append((user, game_id, float(rating)))
# Convert to DataFrame
ratings_df = pd.DataFrame(rows, columns=['user', 'game', 'rating'])   

# Count ratings per user
user_counts = ratings_df['user'].value_counts()

# Filter users with at least 3 ratings
valid_users = user_counts[user_counts >= 3].index

# Filter the dataframe
filtered_ratings_df = ratings_df[ratings_df['user'].isin(valid_users)].copy()

# Step 2: Encode users and games as integer IDs
user_encoder = LabelEncoder()
game_encoder = LabelEncoder()

filtered_ratings_df['user_id'] = user_encoder.fit_transform(filtered_ratings_df['user'])
filtered_ratings_df['game_id'] = game_encoder.fit_transform(filtered_ratings_df['game'])

# Step 3: Create sparse matrix
user_ids = filtered_ratings_df['user_id'].values
game_ids = filtered_ratings_df['game_id'].values
ratings = filtered_ratings_df['rating'].values

ratings_sparse_matrix = coo_matrix((ratings, (user_ids, game_ids)))
print(ratings_sparse_matrix.shape)
# del(filtered_ratings_df)
# del(ratings_df)
# del(rows)

In [ ]:
item_user_matrix = csr_matrix(ratings_sparse_matrix.T)

model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20)
model.fit(item_user_matrix)

# Recommender script testing

In [2]:
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from typing import Dict, List, Tuple
import argparse
import logging
from pathlib import Path
import json

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    handlers=[logging.FileHandler("recommender.log"), logging.StreamHandler()],
)
logger = logging.getLogger(__name__)

In [5]:
# data_dir = Path(__file__).parent.parent.parent / "data" / "crawler"
data_dir = Path().resolve().parent.parent / "data" / "crawler"
ratings_files = list(data_dir.glob("boardgame_ratings_*.parquet"))
if not ratings_files:
    raise FileNotFoundError("No ratings files found")

latest_ratings = max(ratings_files, key=lambda x: x.stat().st_mtime)
logger.info(f"Using ratings file: {latest_ratings}")

2025-06-12 11:48:17,441 - __main__ - INFO - Using ratings file: /home/ubuntu/git/pax_tt_recommender/data/crawler/boardgame_ratings_1749439710.parquet


In [7]:
df_ratings = pd.read_parquet(latest_ratings)

In [49]:
df_ratings = pd.read_parquet("../../data/crawler/boardgame_ratings_1749439709.parquet")
df_ratings.shape

(16146, 22)

In [35]:
df_ratings.shape

(1000, 22)

In [78]:
recommender = GameRecommender(min_ratings_per_user=3, exclude_expansions=False)

In [79]:
recommender.fit(df_ratings)

In [82]:
reverse_mappings_path = "../../data/crawler/reverse_mappings_1749439709.json"
with open(reverse_mappings_path, "w") as f:
    json.dump(recommender.reverse_game_mapping, f)

In [83]:
with open(reverse_mappings_path, "r") as f:
    rgm = json.load(f)
gm = {v: k for k, v in rgm.items()}

In [94]:
# game_embeddings = sparse.load_npz("../../data/crawler/game_embeddings_1749439709.npz")
pos_vec = game_embeddings[recommender.game_mapping[224517]].mean(axis=0)

In [96]:
normalize(np.asarray(pos_vec), norm='l2')

array([[0., 0., 0., ..., 0., 0., 0.]], shape=(1, 363464))

In [93]:
from sklearn.preprocessing import normalize

In [85]:
gm

{179350: '0',
 99808: '1',
 286667: '2',
 170041: '3',
 300993: '4',
 139508: '5',
 103651: '6',
 132326: '7',
 171136: '8',
 170202: '9',
 118337: '10',
 11161: '11',
 4282: '12',
 1693: '13',
 193164: '14',
 2387: '15',
 265684: '16',
 174805: '17',
 7204: '18',
 2127: '19',
 1002: '20',
 4047: '21',
 21613: '22',
 28302: '23',
 7308: '24',
 3149: '25',
 255175: '26',
 24082: '27',
 11385: '28',
 27970: '29',
 298619: '30',
 104805: '31',
 409572: '32',
 156457: '33',
 143994: '34',
 3385: '35',
 2173: '36',
 2047: '37',
 147568: '38',
 285554: '39',
 5585: '40',
 366790: '41',
 5651: '42',
 1720: '43',
 164702: '44',
 233956: '45',
 83040: '46',
 209166: '47',
 158564: '48',
 135888: '49',
 220547: '50',
 292907: '51',
 125368: '52',
 144526: '53',
 275996: '54',
 2150: '55',
 2301: '56',
 303676: '57',
 343525: '58',
 216864: '59',
 302477: '60',
 40107: '61',
 229598: '62',
 29351: '63',
 1678: '64',
 187353: '65',
 1713: '66',
 205831: '67',
 89342: '68',
 20436: '69',
 71099: '7

In [68]:
for k,v in enumerate(df_ratings["id"]):
    break

In [70]:
v

179350

In [66]:
rgm

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 '10': 10,
 '11': 11,
 '12': 12,
 '13': 13,
 '14': 14,
 '15': 15,
 '16': 16,
 '17': 17,
 '18': 18,
 '19': 19,
 '20': 20,
 '21': 21,
 '22': 22,
 '23': 23,
 '24': 24,
 '25': 25,
 '26': 26,
 '27': 27,
 '28': 28,
 '29': 29,
 '30': 30,
 '31': 31,
 '32': 32,
 '33': 33,
 '34': 34,
 '35': 35,
 '36': 36,
 '37': 37,
 '38': 38,
 '39': 39,
 '40': 40,
 '41': 41,
 '42': 42,
 '43': 43,
 '44': 44,
 '45': 45,
 '46': 46,
 '47': 47,
 '48': 48,
 '49': 49,
 '50': 50,
 '51': 51,
 '52': 52,
 '53': 53,
 '54': 54,
 '55': 55,
 '56': 56,
 '57': 57,
 '58': 58,
 '59': 59,
 '60': 60,
 '61': 61,
 '62': 62,
 '63': 63,
 '64': 64,
 '65': 65,
 '66': 66,
 '67': 67,
 '68': 68,
 '69': 69,
 '70': 70,
 '71': 71,
 '72': 72,
 '73': 73,
 '74': 74,
 '75': 75,
 '76': 76,
 '77': 77,
 '78': 78,
 '79': 79,
 '80': 80,
 '81': 81,
 '82': 82,
 '83': 83,
 '84': 84,
 '85': 85,
 '86': 86,
 '87': 87,
 '88': 88,
 '89': 89,
 '90': 90,
 '91': 91,
 '92': 9

In [75]:
df_tmp = df_ratings.set_index("id")
for k,v in df_tmp.iterrows():
    break

In [76]:
k

179350

In [44]:
recommender.rating_matrix

<List of Lists sparse matrix of dtype 'float64'
	with 433290 stored elements and shape (130418, 1000)>

In [77]:
class GameRecommender:
    def __init__(self, min_ratings_per_user: int = 3, exclude_expansions: bool = False):
        """
        Initialize the recommender system.

        Args:
            min_ratings_per_user (int): Minimum number of ratings a user must have to be included
            exclude_expansions (bool): Whether to exclude expansions from recommendations
        """
        self.min_ratings_per_user = min_ratings_per_user
        self.exclude_expansions = exclude_expansions
        self.user_mapping = {}  # Maps user IDs to indices
        self.game_mapping = {}  # Maps game IDs to indices
        self.reverse_game_mapping = {}  # Maps indices back to game IDs
        self.rating_matrix = None
        self.game_embeddings = None
        self.valid_game_ids = (
            None  # Set of valid game IDs (non-expansions if exclude_expansions is True)
        )

    def _create_rating_matrix(self, df: pd.DataFrame) -> sparse.csr_matrix:
        """
        Convert the input dataframe into a sparse rating matrix.

        Args:
            df (pd.DataFrame): Input dataframe with game_id as index and rating columns containing user lists

        Returns:
            sparse.csr_matrix: Sparse matrix of user-game ratings
        """
        # check if the dataframe has a column called "id"
        if "id" in df.columns:
            df = df.set_index("id")

        # Create mappings for users and games
        all_users = set()
        for col in df.columns:
            # Handle both list and non-list values in the ratings
            for users in df[col].dropna():
                if isinstance(users, list):
                    all_users.update(str(user) for user in users)
                else:
                    # If it's a single user, add them directly
                    all_users.add(str(users))

        # Create user mapping
        for i, user in enumerate(sorted(all_users)):
            self.user_mapping[user] = i

        # Create game mapping
        for i, game_id in enumerate(df.index):
            self.game_mapping[game_id] = i
            self.reverse_game_mapping[i] = game_id

#         # Initialize sparse matrix
#         n_users = len(self.user_mapping)
#         n_games = len(self.game_mapping)
#         rating_matrix = sparse.lil_matrix((n_users, n_games))

#         # Fill the rating matrix
#         for game_id, row in df.iterrows():
#             game_idx = self.game_mapping[game_id]
#             for rating, users in row.items():
#                 if rating == "id":  # Skip the id column
#                     continue

#                 # Skip if users is NaN
#                 if users is None:
#                     continue

#                 # Add ratings for each user
#                 for user in users:
#                     if user in self.user_mapping:
#                         user_idx = self.user_mapping[user]
#                         rating_matrix[user_idx, game_idx] = float(rating)

        # Convert to CSR format for efficient operations
        return rating_matrix.tocsr()

    def _filter_users(self, rating_matrix: sparse.csr_matrix) -> sparse.csr_matrix:
        """
        Remove users with fewer than min_ratings_per_user ratings.

        Args:
            rating_matrix (sparse.csr_matrix): Input rating matrix

        Returns:
            sparse.csr_matrix: Filtered rating matrix
        """
        # Count non-zero elements per user (number of ratings)
        user_rating_counts = np.diff(rating_matrix.indptr)

        # Get indices of users with enough ratings
        valid_user_indices = np.where(user_rating_counts >= self.min_ratings_per_user)[
            0
        ]

        # Filter the matrix
        return rating_matrix[valid_user_indices]

    def _load_valid_games(self):
        """Load the set of valid game IDs from the ranks file."""
        if not self.exclude_expansions:
            return None

        # Find the most recent ranks file
        data_dir = Path(__file__).parent.parent.parent / "data" / "crawler"
        ranks_files = list(data_dir.glob("boardgame_ranks_*.csv"))
        if not ranks_files:
            logger.warning("No ranks files found, cannot exclude expansions")
            return None

        latest_ranks = max(ranks_files, key=lambda x: x.stat().st_mtime)
        logger.info(f"Loading valid games from: {latest_ranks}")

        # Read the ranks file and get non-expansion games
        df_ranks = pd.read_csv(latest_ranks, sep="|", escapechar="\\")
        valid_games = df_ranks[df_ranks["is_expansion"] == 0]["id"].tolist()
        return set(str(gid) for gid in valid_games)

    def fit(self, df: pd.DataFrame) -> None:
        """
        Fit the recommender system to the input data.

        Args:
            df (pd.DataFrame): Input dataframe with game_id as index and rating columns containing user lists
        """
        # Create and filter rating matrix
        self.rating_matrix = self._create_rating_matrix(df)
#         self.rating_matrix = self._filter_users(self.rating_matrix)

        # Compute game similarity matrix
#         self.game_embeddings = normalize(self.rating_matrix.T, norm="l2", axis=1)

In [ ]:
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from typing import Dict, List, Tuple
import argparse
import logging
from pathlib import Path
import json

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    handlers=[logging.FileHandler("recommender.log"), logging.StreamHandler()],
)
logger = logging.getLogger(__name__)

class GameRecommender:
    def __init__(self, min_ratings_per_user: int = 3, exclude_expansions: bool = False):
        """
        Initialize the recommender system.
        
        Args:
            min_ratings_per_user (int): Minimum number of ratings a user must have to be included
            exclude_expansions (bool): Whether to exclude expansions from recommendations
        """
        self.min_ratings_per_user = min_ratings_per_user
        self.exclude_expansions = exclude_expansions
        self.user_mapping = {}  # Maps user IDs to indices
        self.game_mapping = {}  # Maps game IDs to indices
        self.reverse_game_mapping = {}  # Maps indices back to game IDs
        self.rating_matrix = None
        self.game_similarity = None
        self.valid_game_ids = None  # Set of valid game IDs (non-expansions if exclude_expansions is True)
        
    def _create_rating_matrix(self, df: pd.DataFrame) -> sparse.csr_matrix:
        """
        Convert the input dataframe into a sparse rating matrix.
        
        Args:
            df (pd.DataFrame): Input dataframe with game_id as index and rating columns containing user lists
            
        Returns:
            sparse.csr_matrix: Sparse matrix of user-game ratings
        """
        # Create mappings for users and games
        all_users = set()
        for col in df.columns:
            if col == 'id':  # Skip the id column
                continue
            # Handle both list and non-list values in the ratings
            for users in df[col].dropna():
                if isinstance(users, list):
                    all_users.update(str(user) for user in users)
                else:
                    # If it's a single user, add them directly
                    all_users.add(str(users))
        
        # Create user mapping
        for i, user in enumerate(sorted(all_users)):
            self.user_mapping[user] = i
            
        # Create game mapping
        for i, game_id in enumerate(df.index):
            self.game_mapping[game_id] = i
            self.reverse_game_mapping[i] = game_id
            
        # Initialize sparse matrix
        n_users = len(self.user_mapping)
        n_games = len(self.game_mapping)
        rating_matrix = sparse.lil_matrix((n_users, n_games))
        
        # Fill the rating matrix
        for game_id, row in df.iterrows():
            game_idx = self.game_mapping[game_id]
            for rating, users in row.items():
                if rating == 'id':  # Skip the id column
                    continue
                    
                # Skip if users is NaN
                if users is None:
                    continue
                    
                # Add ratings for each user
                for user in users:
                    if user in self.user_mapping:
                        user_idx = self.user_mapping[user]
                        rating_matrix[user_idx, game_idx] = float(rating)
        
        # Convert to CSR format for efficient operations
        return rating_matrix.tocsr()
    
    def _filter_users(self, rating_matrix: sparse.csr_matrix) -> sparse.csr_matrix:
        """
        Remove users with fewer than min_ratings_per_user ratings.
        
        Args:
            rating_matrix (sparse.csr_matrix): Input rating matrix
            
        Returns:
            sparse.csr_matrix: Filtered rating matrix
        """
        # Count non-zero elements per user (number of ratings)
        user_rating_counts = np.diff(rating_matrix.indptr)
        
        # Get indices of users with enough ratings
        valid_user_indices = np.where(user_rating_counts >= self.min_ratings_per_user)[0]
        
        # Filter the matrix
        return rating_matrix[valid_user_indices]

    def fit(self, df: pd.DataFrame) -> None:
        """
        Fit the recommender system to the input data.
        
        Args:
            df (pd.DataFrame): Input dataframe with game_id as index and rating columns containing user lists
        """
        # Create and filter rating matrix
        self.rating_matrix = self._create_rating_matrix(df)
        self.rating_matrix = self._filter_users(self.rating_matrix)
        
        # Compute game similarity matrix
        self.game_similarity = cosine_similarity(self.rating_matrix.T)
        
    def recommend_similar_games(self, 
                              game_ids: List[str], 
                              disliked_games: List[str] = None,
                              n_recommendations: int = 5,
                              anti_weight: float = 1.0) -> List[Tuple[str, float]]:
        """
        Generate recommendations based on a list of game IDs, with optional anti-recommendations.
        
        Args:
            game_ids (List[str]): List of game IDs to base recommendations on
            disliked_games (List[str], optional): List of game IDs to use as anti-recommendations
            n_recommendations (int): Number of recommendations to generate
            anti_weight (float): Weight to apply to anti-recommendations
            
        Returns:
            List[Tuple[str, float]]: List of (game_id, similarity_score) tuples
        """
        # Convert liked game IDs to indices
        game_indices = []
        for game_id in game_ids:
            if game_id in self.game_mapping:
                game_indices.append(self.game_mapping[game_id])
        
        if not game_indices:
            return []
            
        # Calculate average similarity scores for all games
        avg_similarities = np.zeros(len(self.game_mapping))
        for game_idx in game_indices:
            avg_similarities += self.game_similarity[game_idx]
        avg_similarities /= len(game_indices)
        
        # Apply anti-recommendations if provided
        if disliked_games:
            disliked_indices = []
            for game_id in disliked_games:
                if game_id in self.game_mapping:
                    disliked_indices.append(self.game_mapping[game_id])
            
            if disliked_indices:
                # Calculate average similarity to disliked games
                anti_similarities = np.zeros(len(self.game_mapping))
                for game_idx in disliked_indices:
                    anti_similarities += self.game_similarity[game_idx]
                anti_similarities /= len(disliked_indices)
                
                # Subtract anti-similarities from the main similarities
                avg_similarities -= anti_similarities * anti_weight
        
        # Remove the input games from recommendations
        for game_idx in game_indices:
            avg_similarities[game_idx] = -1
        if disliked_games:
            for game_idx in disliked_indices:
                avg_similarities[game_idx] = -1

        # Exclude expansions if configured
        if self.exclude_expansions and self.valid_game_ids:
            for idx, game_id in self.reverse_game_mapping.items():
                if game_id not in self.valid_game_ids:
                    avg_similarities[idx] = -1
            
        # Get top N similar games
        top_indices = np.argsort(avg_similarities)[-n_recommendations:][::-1]
        
        return [(self.reverse_game_mapping[idx], avg_similarities[idx]) 
                for idx in top_indices if avg_similarities[idx] > 0]





In [46]:
game_embeddings = sparse.load_npz("../../data/crawler/game_embeddings_1749439709.npz")

In [47]:
type(game_embeddings)

scipy.sparse._csr.csr_matrix